# GWP # 1

s101 MScFE 560 Financial Markets 

Jaepil Choi

## My Portfolio: Portfolio A

Buy 1 stock and short 1 stock

In [77]:
from typing import Tuple

import pandas as pd
import numpy as np

from itertools import permutations

## Step 1

For each investment, you are given statistics in this .csv file the stock’s.
You also have the correlation and covariance matrix of historical returns (download this
.csv file)

1. Average return
2. Volatility
3. Skewness
4. Kurtosis

You must compute the portfolio’s average return and portfolio volatility. You do not
need to compute the portfolio skewness and portfolio kurtosis.

In [78]:
statistics_df = pd.read_csv('MScFE560_FM_GWP1_Data.csv', nrows=4, index_col=0)
statistics_df = statistics_df.iloc[:, :5]
statistics_df

,AAPL,AMZN,NFLX,META,GOOG
Mean,0.044858,0.010362,-0.029675,-0.006934,0.029035
St Dev,0.009320,0.009557,0.013409,0.011582,0.008376
Skew,-0.335173,-0.312865,-3.101654,-1.936506,-0.197700
Kurt,5.034093,4.729806,41.968015,23.289236,4.310171


In covariance matrix, `GOOGL` (Alphabet Inc Class A) is given instead of `GOOG` (Alphabet Inc Class C). 

To match the name, I changed `GOOGL` to `GOOG`

In [99]:
cov_matrix_df = pd.read_csv('MScFE560_FM_GWP1_Data.csv', skiprows=5, nrows=6, header=1, index_col=0)
cov_matrix_df = cov_matrix_df.iloc[:, :5]
cov_matrix_df.rename({'GOOGL': 'GOOG'}, axis=0, inplace=True)
cov_matrix_df.rename({'GOOGL': 'GOOG'}, axis=1, inplace=True)
cov_matrix_df

,AAPL,AMZN,NFLX,META,GOOG
AAPL,1.000000,0.660739,0.460041,0.594785,0.698431
AMZN,0.660739,1.000000,0.593812,0.626198,0.679323
NFLX,0.460041,0.593812,1.000000,0.515892,0.492697
META,0.594785,0.626198,0.515892,1.000000,0.668024
GOOG,0.698431,0.679323,0.492697,0.668024,1.000000


### 1-1 Portfolio's average return

In [100]:
mean = statistics_df.loc['Mean', :] 
std = statistics_df.loc['St Dev', :]

In [101]:
sid_list = statistics_df.columns
sid_list

Index(['AAPL', 'AMZN', 'NFLX', 'META', 'GOOG'], dtype='object')

Since I can long 1 stock and short 1 stock, there are $ 2^5 $ number of cases

In [118]:
longshort_cases = permutations(sid_list, 2)
longshort_cases = list(longshort_cases)

In [103]:
def get_portfolio_avg_return(case: tuple) -> Tuple[tuple, float]:
    long_sid, short_sid = case
    
    long_avg_return = mean.loc[long_sid]
    short_avg_return = -mean.loc[short_sid]

    returns = np.array([long_avg_return, short_avg_return])
    weights = np.array([0.5, 0.5])

    portfolio_avg_returns = np.dot(weights, returns)

    return (case, portfolio_avg_returns)

In [104]:
longshort_portfolio_returns = []

for case in longshort_cases:
    longshort_portfolio_returns.append(get_portfolio_avg_return(case))

In [105]:
longshort_portfolio_returns = sorted(longshort_portfolio_returns, key=lambda x: x[1], reverse=True)

In [106]:
# Top 5 return portfolios
longshort_portfolio_returns[:5]

[(('AAPL', 'NFLX'), 0.0372663865),
 (('GOOG', 'NFLX'), 0.029354981500000002),
 (('AAPL', 'META'), 0.025896114499999998),
 (('AMZN', 'NFLX'), 0.020018404),
 (('GOOG', 'META'), 0.0179847095)]

In [107]:
# Bottom 5 return portfolios
longshort_portfolio_returns[-5:]

[(('META', 'GOOG'), -0.0179847095),
 (('NFLX', 'AMZN'), -0.020018404),
 (('META', 'AAPL'), -0.025896114499999998),
 (('NFLX', 'GOOG'), -0.029354981500000002),
 (('NFLX', 'AAPL'), -0.0372663865)]

### 1-2 Portfolio's volatility

In [108]:
def get_portfolio_variance(case: tuple) -> Tuple[tuple, float]:
    long_sid, short_sid = case
    
    long_std = std.loc[long_sid]
    short_std = std.loc[short_sid]

    covariance = cov_matrix_df.loc[long_sid, short_sid]

    stds = np.array([long_std, short_std])
    weights = np.array([0.5, 0.5])

    portfolio_variance = np.dot(weights ** 2, stds ** 2) + 2 * covariance * np.prod(weights) 

    return (case, portfolio_variance)

In [110]:
longshort_portfolio_vars = []

for case in longshort_cases:
    longshort_portfolio_vars.append(get_portfolio_variance(case))

In [111]:
longshort_portfolio_vars = sorted(longshort_portfolio_vars, key=lambda x: x[1], reverse=True)

In [112]:
# Top 5 variance portfolios
longshort_portfolio_vars[:5]

[(('AAPL', 'GOOG'), 0.34925488359180457),
 (('GOOG', 'AAPL'), 0.34925488359180457),
 (('AMZN', 'GOOG'), 0.33970171681053274),
 (('GOOG', 'AMZN'), 0.33970171681053274),
 (('META', 'GOOG'), 0.3340629657278744)]

In [113]:
# Botoom 5 variance portfolios
longshort_portfolio_vars[-5:]

[(('META', 'NFLX'), 0.25802452796054925),
 (('NFLX', 'GOOG'), 0.24641102899422132),
 (('GOOG', 'NFLX'), 0.24641102899422132),
 (('AAPL', 'NFLX'), 0.2300872373244794),
 (('NFLX', 'AAPL'), 0.2300872373244794)]

### Sharpe ratio

In [116]:
port_returns_dict = dict(longshort_portfolio_returns)
port_vars_dict = dict(longshort_portfolio_vars)

In [122]:
longshort_portfolio_sharpe = []

for case in longshort_cases:
    ret = port_returns_dict[case]
    std = np.sqrt(port_vars_dict[case])

    sharpe = ret / std

    longshort_portfolio_sharpe.append((case, sharpe))

In [124]:
longshort_portfolio_sharpe = sorted(longshort_portfolio_sharpe, key=lambda x: x[1], reverse=True)

In [125]:
# Top 5 variance portfolios
longshort_portfolio_sharpe[:5]

[(('AAPL', 'NFLX'), 0.07769105502196288),
 (('GOOG', 'NFLX'), 0.059135972039229295),
 (('AAPL', 'META'), 0.0474820303360983),
 (('AMZN', 'NFLX'), 0.03673418114823933),
 (('GOOG', 'META'), 0.031116393935357714)]

### Answer

I'll choose `('AAPL', 'NFLX')` since it has the highest sharpe ratio

In [129]:
portfolio = ('AAPL', 'NFLX')

print(f'Portfolio: Long 1 {portfolio[0]} / Short 1 {portfolio[1]}')
print(f'Portfolio return: {port_returns_dict[portfolio]:.3f}')
print(f'Portfolio std: {np.sqrt(port_vars_dict[portfolio]):.3f}')

Portfolio: Long 1 AAPL / Short 1 NFLX
Portfolio return: 0.037
Portfolio std: 0.480


## Step 2

Answer the following questions:

Each Team Member/Portfolio Manager will answer the following questions about their
own portfolio:

### 2-1 Shorting

#### a. Can this portfolio be shorted? (Hint: Yes, but be sure to explain part b)

## Step 3